# Pooling

### Maximum Pooling and Average Pooling

For pooling layers, no kernel is needed, computing the output as either the maximum or the average of each region in the input.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [5]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y

Validate the output of the two-dimensional max-pooling layer.

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

Validate the output of the two-dimensional average-pooling layer.

In [6]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## Padding and Stride

As with convolutional layers, pooling layers
change the output shape.
And as before, we can adjust the operation to achieve a desired output shape
by padding the input and adjusting the stride.
We can demonstrate the use of padding and strides
in pooling layers via the built-in two-dimensional max-pooling layer from the deep learning framework.
We first construct an input tensor `X` whose shape has four dimensions,
where the number of examples (batch size) and number of channels are both 1.

In [12]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4)) # (batch_size, channel, height, width)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

Since pooling aggregates information from an area, **deep learning frameworks default to matching pooling window sizes and stride**. For instance, if we use a pooling window of shape (3, 3) we get a stride shape of (3, 3) by default.

In [13]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

The stride and padding can be manually specified to override framework defaults if needed.

In [14]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

We can specify an arbitrary rectangular pooling window with arbitrary height and width respectively.

In [16]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

## Multiple Channels

When processing multi-channel input data,
**the pooling layer pools each input channel separately**,
rather than summing the inputs up over channels
as in a convolutional layer.
This means that the number of output channels for the pooling layer
is the same as the number of input channels.

Below, we will concatenate tensors `X` and `X + 1`
on the channel dimension to construct an input with 2 channels.

In [17]:
X = torch.cat((X, X + 1), 1) # NOTE: here 1 means adding channel since (batch_size, channel, height, width)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [18]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2) # kernel _size=3
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])